Tutorial Source: https://leafmap.org/maplibre/live_update_feature/

In [6]:
#uncomment if need to install package
#%pip install "leafmap[maplibre]"

In [8]:
#import packages
import os
import time
import requests
import leafmap.maplibregl as leafmap

In [83]:
#set key
os.environ["MAPTILER_KEY"] = "YOUR_API_KEY"

In [12]:
MAPTILER_KEY = leafmap.get_api_key("MAPTILER_KEY")
style = f"https://api.maptiler.com/maps/streets/style.json?key={MAPTILER_KEY}"

In [45]:
m = leafmap.Map(center=[-100, 40], zoom=3, style=style)
url = "https://github.com/opengeos/datasets/releases/download/us/arc_with_bearings.geojson"
geojson = requests.get(url).json()
#extract bearings
bearings = geojson["features"][0]["properties"]["bearings"]
#extract coordinates
coordinates = geojson["features"][0]["geometry"]["coordinates"][:-1]
#add geojson route to map 
m.add_geojson(geojson, name="route")

origin = [-122.414, 37.776] #SF
destination = [-77.032, 38.913] #DC

#creates a GeoJSON FeatureCollection containing a single point feature at the origin coordinates
point = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {"type": "Point", "coordinates": origin},
        }
    ],
}

#defines a source for the map using the created GeoJSON point
source = {"type": "geojson", "data": point}
m.add_source("point", source)

#define and add layer for point
layer = {
    "id": "point",
    "source": "point",
    "type": "symbol",
    "layout": {
        "icon-image": "airport_15",
        "icon-rotate": ["get", "bearing"],
        "icon-rotation-alignment": "map",
        "icon-overlap": "always",
        "icon-ignore-placement": True,
    },
}
m.add_layer(layer)
m

Map(height='600px', map_options={'bearing': 0, 'center': (-100, 40), 'pitch': 0, 'style': 'https://api.maptile…

In [48]:
for index, coordinate in enumerate(coordinates):
    point["features"][0]["geometry"]["coordinates"] = coordinate
    point["features"][0]["properties"]["bearing"] = bearings[index]
    m.set_data("point", point)
    time.sleep(0.05)

Tutorial Source: https://leafmap.org/maplibre/live_update_feature/

In [36]:
#uncomment if need to install package
#%pip install "leafmap[maplibre]"

In [34]:
import os
import time
import geopandas as gpd
import leafmap.maplibregl as leafmap

In [85]:
#set key
os.environ["MAPTILER_KEY"] = "YOUR_API_KEY"

In [75]:
m = leafmap.Map(center=[-122.019807, 45.632433], zoom=14, pitch=60, style="3d-terrain")
m

Map(height='600px', map_options={'bearing': 0, 'center': (-122.019807, 45.632433), 'pitch': 60, 'style': {'ver…

In [77]:
url = "https://maplibre.org/maplibre-gl-js/docs/assets/hike.geojson"
gdf = gpd.read_file(url)
coordinates = list(gdf.geometry[0].coords)
print(coordinates[:5])

[(-122.019807, 45.632433), (-122.019767, 45.632453), (-122.01971, 45.632472), (-122.01971, 45.632505), (-122.019739, 45.632531)]


In [79]:
source = {
    "type": "geojson",
    "data": {
        "type": "Feature",
        "geometry": {"type": "LineString", "coordinates": [coordinates[0]]},
    },
}
m.add_source("trace", source)
layer = {
    "id": "trace",
    "type": "line",
    "source": "trace",
    "paint": {"line-color": "yellow", "line-opacity": 0.75, "line-width": 5},
}
m.add_layer(layer)
m.jump_to({"center": coordinates[0], "zoom": 14})
m.set_pitch(30)

In [81]:
for coord in coordinates:
    time.sleep(0.005)
    source["data"]["geometry"]["coordinates"].append(coord)
    m.set_data("trace", source["data"])
    m.pan_to(coord)